In [ ]:
!pip install textblob
!pip install nltk
!pip install gensim

In [ ]:
!python -m textblob.download_corpora lite

In [18]:
import pandas as pd
from textblob import TextBlob
import nltk
import gensim
import numpy as np
import re

In [ ]:
df = pd.read_csv('EarningsDataSentences Dojo.csv')

In [9]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/nbuser/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
stop_en = stopwords.words('english')

In [6]:
df.head(10)

,ID,Sentence,Keyword,Health,Climate,Inequality,Pollution,Waste,Poverty,Classification,cleanedsentence
0,2028351,We believe we could offer greater performance ...,batteries,0,0,0,1,0,0,0,We believe we could offer greater performance ...
1,2028351,""", 'Meanwhile, many other companies continue t...",batteries,0,0,0,1,0,0,0,Meanwhile many other companies continue to te...
2,2028351,Our batteries are a drop-in replacement and re...,batteries,0,0,0,1,0,0,0,Our batteries are a dropin replacement and ret...
3,2028351,"', 'We are also continuing to work with our OE...",batteries,0,0,0,1,0,0,1,We are also continuing to work with our OEM p...
4,2028351,We're managing our working capital very carefu...,health,1,0,0,0,0,0,0,Were managing our working capital very careful...
5,2028351,"', 'We began an intensive product development ...",batteries,0,0,0,1,0,0,1,We began an intensive product development per...
6,2028351,Walmart spent five months testing our batterie...,batteries,0,0,0,1,0,0,0,Walmart spent five months testing our batterie...
7,2028351,Then the company had the materials handling ve...,batteries,0,0,0,1,0,0,0,Then the company had the materials handling ve...
8,2028351,"Typically, they test our batteries in their mo...",batteries,0,0,0,1,0,0,0,Typically they test our batteries in their mos...
9,2028351,"', ""They also ask for a confirmation from the ...",batteries,0,0,0,1,0,0,0,They also ask for a confirmation from the mat...


In [15]:
df_train = df.iloc[:15000,:]

In [16]:
def clean_text(text):
    t = TextBlob(text.lower()).tokenize()
    t = [w for w in t if re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', w) and w not in stop_en]
    t = t.lemmatize()
    return " ".join(t)

In [73]:
df_train['txt2'] = df['cleanedsentence'].apply(lambda x: clean_text(x))

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [ ]:
list(df_train['txt_nouns'].tolist())

In [ ]:
#global_words = pd.Series(" ".join(df_train['txt2']).split()).value_counts()
global_words

In [76]:
common_words = global_words[(global_words < 5) | (global_words > 500)]

In [77]:
def remove_common_words(text):
    t = TextBlob(text).tokenize()
    t = [w for w in t if w not in common_words]
    return t

In [78]:
df_train['txt2'] = df_train['txt2'].apply(lambda x: remove_common_words(x))

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [79]:
data_text = list(df_train['txt2'])

In [80]:
bigram = gensim.models.Phrases(data_text, min_count=5, threshold=20) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_text], threshold=20)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [81]:
data_text = [trigram_mod[bigram_mod[doc]] for doc in data_text]

# Sentence Clustering

In [83]:
from gensim import corpora
dictionary = corpora.Dictionary(data_text)
corpus = [dictionary.doc2bow(text) for text in data_text]

In [84]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 6, id2word=dictionary, passes=15, random_state=100, alpha='auto')
ldamodel.save('model10.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.013*"fourth" + 0.012*"portfolio" + 0.011*"lower" + 0.011*"significant"')
(1, '0.013*"time" + 0.013*"like" + 0.012*"thats" + 0.012*"really"')
(2, '0.018*"strategy" + 0.009*"grew" + 0.008*"pleased" + 0.008*"clients"')
(3, '0.012*"continued" + 0.012*"demand" + 0.011*"product" + 0.011*"performance"')
(4, '0.012*"project" + 0.012*"data" + 0.010*"safety" + 0.008*"water"')
(5, '0.013*"company" + 0.012*"solar" + 0.011*"services" + 0.011*"ecosystem"')


In [63]:
!pip install pyldavis

    100% |████████████████████████████████| 1.6MB 1.3MB/s ta 0:00:01    19% |██████                          | 307kB 1.3MB/s eta 0:00:02
  Running setup.py bdist_wheel for pyldavis ... done
  Stored in directory: /home/nbuser/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyldavis


In [85]:
lda = gensim.models.ldamodel.LdaModel.load('model10.gensim')

import pyLDAvis.gensim
pyLDAvis.enable_notebook()
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [102]:
results = []
for i in range(len(data_text)):
    results.append(ldamodel.get_document_topics(dictionary.doc2bow(data_text[i])))

In [105]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [107]:
vectorizer = TfidfVectorizer()

In [112]:
vectorizer.fit(df_train['txt2'].apply(lambda x: " ".join(x)))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)